# Benchmarking Different Quantum Hardware Devices using Grover's Algorithm

## Code Steps:
1. [Libraries, Parameters and Global Variables](#variables)
2. [Constructing Grover's Circuit using Qiskit](#grover_circuit)
3. [Running Grover's Circuits on IBM Hardware](#ibm)
4. [Running Grover's Circuits on AWS Braket](#aws_circuits)
    1. [Running Grover's Circuit on IonQ (Forte)](#ionq)
    2. [Running Grover's Circuit on IQM (Garnet)](#iqm)
5. [Collecting Results from Queued Jobs/Tasks](#results)
5. [Saving Data to CSV and JSON files](#save_data)

## 1. Libraries, Parameters and Global Variables <a name="variables"></a>

In [1]:
# Built-in modules
from datetime import datetime
import numpy as np
import os
import pandas as pd

# Imports from Braket
from qiskit_braket_provider import BraketProvider

# Imports from Qiskit
from qiskit import qasm3

# Imports from Qiskit Runtime
from qiskit_ibm_runtime import QiskitRuntimeService

# Import helper functions
from helper import generate_short_key


In [2]:
## Initialize global variables to store input parameters and output

# Initialize the DataFrame for input parameters
param_df = pd.DataFrame(columns=['key', 'target_states', 'quantum_circuit', 'num_qubits', 'depth'])

# Initialize the DataFrame for storing output
output_df = pd.DataFrame(columns=['key', 'device_name', 'transpiled_depth', 'job_id', 'counts', 'execution_time'])

print(param_df)
print(output_df)

# Sample key generation
key = generate_short_key()
print(f"\nGenerated Key: {key}")

Empty DataFrame
Columns: [key, target_states, quantum_circuit, num_qubits, depth]
Index: []
Empty DataFrame
Columns: [key, device_name, transpiled_depth, job_id, counts, execution_time]
Index: []

Generated Key: b357abaa


In [3]:
# Read Data from JSON files to DataFrames
## Do this for existing data only!
from helper import get_quantum_circuit_from_json

param_json = 'saved_data/parameters_20241208_1221.json'
output_json = 'saved_data/output_20241208_1221.json'

param_df = pd.read_json(param_json, orient="records")
param_df['quantum_circuit'] = get_quantum_circuit_from_json(param_df['quantum_circuit'])

output_df = pd.read_json(output_json, orient="records")

In [4]:
param_df

,key,target_states,quantum_circuit,num_qubits,depth
0,e0dcdfc9,[01],"((Instruction(name='h', num_qubits=1, num_clbi...",2,12
1,23caeab0,[111],"((Instruction(name='h', num_qubits=1, num_clbi...",3,18
2,a00dc22c,[1001],"((Instruction(name='h', num_qubits=1, num_clbi...",4,27
3,cdbc3c51,"[00111, 10011]","((Instruction(name='h', num_qubits=1, num_clbi...",5,38
4,0e14030c,"[011011, 101001]","((Instruction(name='h', num_qubits=1, num_clbi...",6,50
5,26c7430b,[01],"((Instruction(name='h', num_qubits=1, num_clbi...",2,12
6,7a02b35f,[111],"((Instruction(name='h', num_qubits=1, num_clbi...",3,18
7,dfa97b52,[1001],"((Instruction(name='h', num_qubits=1, num_clbi...",4,27
8,52ca2cc5,"[00111, 10011]","((Instruction(name='h', num_qubits=1, num_clbi...",5,38
9,afb87e2b,"[011011, 101001]","((Instruction(name='h', num_qubits=1, num_clbi...",6,50


In [5]:
output_df

,key,device_name,transpiled_depth,job_id,counts,execution_time
0,e0dcdfc9,aer_sim,6,,{'01': 1024},1970-01-01 00:18:06
1,23caeab0,aer_sim,10,,"{'111': 969, '001': 9, '110': 12, '011': 11, '...",1970-01-01 00:05:37
2,a00dc22c,aer_sim,14,,"{'1001': 986, '1101': 5, '0011': 3, '1100': 6,...",1970-01-01 00:03:21
3,cdbc3c51,aer_sim,20,,"{'00111': 505, '10011': 483, '10010': 2, '0110...",1970-01-01 00:05:24
4,0e14030c,aer_sim,26,,"{'011011': 507, '101001': 513, '111011': 1, '0...",1970-01-01 00:03:02
5,e0dcdfc9,aer_noisy,12,,"{'01': 950, '11': 49, '00': 19, '10': 6}",1970-01-01 04:16:30
6,23caeab0,aer_noisy,176,,"{'000': 28, '111': 788, '101': 37, '110': 38, ...",1970-01-01 03:22:52
7,a00dc22c,aer_noisy,659,,"{'1011': 40, '1001': 427, '0110': 37, '0111': ...",1970-01-01 02:24:09
8,cdbc3c51,aer_noisy,2332,,"{'10000': 32, '11111': 37, '00110': 40, '11010...",1970-01-01 03:27:55
9,0e14030c,aer_noisy,6315,,"{'000011': 19, '111110': 13, '100110': 14, '11...",1970-01-01 02:33:09


## 2. Constructing Grover's Circuit using Qiskit <a name="grover_circuit"></a>
1. This section takes an upper limit to the number of qubits $(n_{max})$ in a circuit, describing the size of the database. 
2. A random string of n-bit `target_states`, i.e., states to be searched in the database, is generated to construct the Grover's Oracle. To reduce circuit depth, we set the number of possible target_states to be atmost 4.
3. A total of $n_{max}$ Grover's circuits are generated, each with increasing number of qubits from 2 to $n_{max}$.
4. This list of target states and Grover's circuits are stored in the `param_df` DataFrame.

In [6]:
# Load target states if previously exist
target_states = param_df['target_states']
print(f"Target states: \n {target_states}")

Target states: 
 0                [01]
1               [111]
2              [1001]
3      [00111, 10011]
4    [011011, 101001]
5                [01]
6               [111]
7              [1001]
8      [00111, 10011]
9    [011011, 101001]
Name: target_states, dtype: object


In [ ]:
from helper import generate_random_n_bit_strings

# Maximum number of qubits
n_max = 6

# Generate random target_states for the Oracle
target_states = []

for n in range(2, n_max+1):
    states = generate_random_n_bit_strings(n)
    target_states.append(states)
print(f"Target states: \n {target_states}")

In [14]:
from grover import grover_circuit

# Generate n_max Grover's circuits and storing in Data Frame

# Drop the existing dataframe is not required
# param_df.drop(param_df.index, inplace=True)

for states in target_states:
    circuit = grover_circuit(states)
    key = generate_short_key()
    param_df.loc[len(param_df)] = [key, list(states), circuit, circuit.num_qubits, circuit.depth()]

param_df

,key,target_states,quantum_circuit,num_qubits,depth
0,e0dcdfc9,[01],"((Instruction(name='h', num_qubits=1, num_clbi...",2,12
1,23caeab0,[111],"((Instruction(name='h', num_qubits=1, num_clbi...",3,18
2,a00dc22c,[1001],"((Instruction(name='h', num_qubits=1, num_clbi...",4,27
3,cdbc3c51,"[00111, 10011]","((Instruction(name='h', num_qubits=1, num_clbi...",5,38
4,0e14030c,"[011011, 101001]","((Instruction(name='h', num_qubits=1, num_clbi...",6,50
5,26c7430b,[01],"((Instruction(name='h', num_qubits=1, num_clbi...",2,12
6,7a02b35f,[111],"((Instruction(name='h', num_qubits=1, num_clbi...",3,18
7,dfa97b52,[1001],"((Instruction(name='h', num_qubits=1, num_clbi...",4,27
8,52ca2cc5,"[00111, 10011]","((Instruction(name='h', num_qubits=1, num_clbi...",5,38
9,afb87e2b,"[011011, 101001]","((Instruction(name='h', num_qubits=1, num_clbi...",6,50


In [ ]:
# Print the Grover's circuits
for qc in param_df["quantum_circuit"]:
    print(qc)

global phase: π
        ┌───┐      1-Z ┌───┐┌───┐               ┌───┐┌───┐      ░ ┌─┐   
   q_0: ┤ H ├───────■──┤ H ├┤ X ├────────────■──┤ X ├┤ H ├──────░─┤M├───
        ├───┤┌───┐  │  ├───┤├───┤┌───┐┌───┐┌─┴─┐├───┤├───┤┌───┐ ░ └╥┘┌─┐
   q_1: ┤ H ├┤ X ├──■──┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├─░──╫─┤M├
        └───┘└───┘     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘ ░  ║ └╥┘
meas: 2/═══════════════════════════════════════════════════════════╩══╩═
                                                                   0  1 
        ┌───┐ 1-Z ┌───┐┌───┐          ┌───┐┌───┐      1-Z ┌───┐┌───┐          »
   q_0: ┤ H ├──■──┤ H ├┤ X ├───────■──┤ X ├┤ H ├───────■──┤ H ├┤ X ├───────■──»
        ├───┤  │  ├───┤├───┤       │  ├───┤├───┤       │  ├───┤├───┤       │  »
   q_1: ┤ H ├──■──┤ H ├┤ X ├───────■──┤ X ├┤ H ├───────■──┤ H ├┤ X ├───────■──»
        ├───┤  │  ├───┤├───┤┌───┐┌─┴─┐├───┤├───┤┌───┐  │  ├───┤├───┤┌───┐┌─┴─┐»
   q_2: ┤ H ├──■──┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├──■──┤ H ├┤ X ├┤ H ├

## 3. Running Grover's Circuits on IBM Hardware <a name="ibm"></a>

In [ ]:
# Save an IBM Quantum account as the default account, and load saved credentials
from ibm_qiskit.config import api_key 

QiskitRuntimeService.save_account(
    channel="ibm_quantum", token=api_key, set_as_default=True, overwrite=True
)
service = QiskitRuntimeService()
# service.backends()  # List all available backends

In [ ]:
from ibm_qiskit.run import aer_without_noise, aer_noisy

key_circuit = list(zip(param_df['key'], param_df['quantum_circuit']))

# Run circuits on Aer simulator (without noise)
output = aer_without_noise(key_circuit)
for key, depth, counts, time in output:
    output_df.loc[len(output_df)] = [key, 'aer_sim', depth, '', counts, time]
    

# Select an IBM backend device to model noise
backend = service.backend("ibm_fez")    # Choosing 'ibm_fez' backend 
print(f"Backend Selected for noise model simulation: {backend}")

# Run circuits on noisy Aer simulator
output = aer_noisy(key_circuit, backend)
for key, depth, counts, time in output:
    output_df.loc[len(output_df)] = [key, 'aer_noisy', depth, '', counts, time]

In [ ]:
# Select an IBM backend device

backends = [service.backend("ibm_fez"), service.backend("ibm_torino"), 
            service.backend("ibm_kyiv"), service.backend("ibm_brisbane")] 
print(f"Backends Selected: {backends}")

In [ ]:
from ibm_qiskit.run import run_backend_job

key_circuit = list(zip(param_df['key'], param_df['quantum_circuit']))

# Run circuits on actual IBM backends
for backend in backends:
    ibm_device = backend.name
    output = run_backend_job(key_circuit, backend)
    for key, job_id, depth in output:
        output_df.loc[len(output_df)] = [key, ibm_device, depth, job_id, '', '']

In [18]:
# Print all output entries for IBM Hardware
ibm_df = output_df[output_df['device_name'].str.startswith(('aer', 'ibm'))]
ibm_df

,key,device_name,transpiled_depth,job_id,counts,execution_time
0,e0dcdfc9,aer_sim,6,,{'01': 1024},1970-01-01 00:00:01.086991072
1,23caeab0,aer_sim,10,,"{'111': 969, '001': 9, '110': 12, '011': 11, '...",1970-01-01 00:00:00.337097883
2,a00dc22c,aer_sim,14,,"{'1001': 986, '1101': 5, '0011': 3, '1100': 6,...",1970-01-01 00:00:00.201308966
3,cdbc3c51,aer_sim,20,,"{'00111': 505, '10011': 483, '10010': 2, '0110...",1970-01-01 00:00:00.324103832
4,0e14030c,aer_sim,26,,"{'011011': 507, '101001': 513, '111011': 1, '0...",1970-01-01 00:00:00.182510853
5,e0dcdfc9,aer_noisy,12,,"{'01': 950, '11': 49, '00': 19, '10': 6}",1970-01-01 00:00:15.390370369
6,23caeab0,aer_noisy,176,,"{'000': 28, '111': 788, '101': 37, '110': 38, ...",1970-01-01 00:00:12.172620296
7,a00dc22c,aer_noisy,659,,"{'1011': 40, '1001': 427, '0110': 37, '0111': ...",1970-01-01 00:00:08.649165869
8,cdbc3c51,aer_noisy,2332,,"{'10000': 32, '11111': 37, '00110': 40, '11010...",1970-01-01 00:00:12.475147247
9,0e14030c,aer_noisy,6315,,"{'000011': 19, '111110': 13, '100110': 14, '11...",1970-01-01 00:00:09.189412594


## 4. Running Grover's Circuits on AWS Braket <a name="aws_circuits"></a>

In [19]:

# Use Braket SDK Cost Tracking to estimate the cost to run this task
from braket.tracking import Tracker

t = Tracker().start()

In [20]:
# Get AWS provider names
provider = BraketProvider()
print(provider.backends())

[BraketBackend[Ankaa-2], BraketBackend[Aria 1], BraketBackend[Aria 2], BraketBackend[Forte 1], BraketBackend[Garnet], BraketBackend[SV1], BraketBackend[dm1]]


In [ ]:
from aws_braket.run import aws_local_simulator, aws_online_simulator

key_circuit = list(zip(param_df['key'], param_df['quantum_circuit']))

# Run circuits on AWS local simulator (zero noise model)
output = aws_local_simulator(key_circuit)
for key, depth, counts, time in output:
    output_df.loc[len(output_df)] = [key, 'aws_local', depth, '', counts, time]


# Run circuits on noisy AWS StateVector simulator
sim = "SV1"
output = aws_online_simulator(key_circuit, sim)
for key, depth, task_arn, counts, time in output:
    device = 'aws_'+sim
    output_df.loc[len(output_df)] = [key, device, depth, task_arn, counts, time]

### A. Running Grover's circuits on IonQ (Forte) <a name="ionq"></a>

In [ ]:
from aws_braket.run import aws_run_task

key_circuit = list(zip(param_df['key'], param_df["quantum_circuit"]))
ionq_device = "Forte 1"
shots = 100

# Run circuits on AWS linked IonQ hardware
output = aws_run_task(key_circuit, ionq_device, shots)
for key, depth, task_arn in output:
    device = 'aws_'+ionq_device
    output_df.loc[len(output_df)] = [key, device, depth, task_arn, '', '']

c:\Users\Pranavi\ee520\venv\Lib\site-packages\qiskit_braket_provider\providers\adapter.py:457: UserWarning: The Qiskit circuit contains barrier instructions that are ignored.
  warnings.warn(
c:\Users\Pranavi\ee520\venv\Lib\site-packages\qiskit_braket_provider\providers\adapter.py:500: UserWarning: Device does not support global phase; global phase of 3.141592653589793 will not be included in Braket circuit
  warnings.warn(
c:\Users\Pranavi\ee520\venv\Lib\site-packages\qiskit_braket_provider\providers\adapter.py:457: UserWarning: The Qiskit circuit contains barrier instructions that are ignored.
  warnings.warn(
c:\Users\Pranavi\ee520\venv\Lib\site-packages\qiskit_braket_provider\providers\adapter.py:500: UserWarning: Device does not support global phase; global phase of 3.141592653589793 will not be included in Braket circuit
  warnings.warn(
c:\Users\Pranavi\ee520\venv\Lib\site-packages\qiskit_braket_provider\providers\adapter.py:457: UserWarning: The Qiskit circuit contains barrier 

An error occurred on key cdbc3c51:
 An error occurred (ValidationException) when calling the CreateQuantumTask operation: Number of 1-qubit gates should be <=950, but 1052 specified for the device arn:aws:braket:us-east-1::device/qpu/ionq/Forte-1


### B. Running Grover's circuits on IQM (Garnet) <a name="iqm"></a>

In [ ]:
from aws_braket.run import aws_run_task

key_circuit = list(zip(param_df['key'], param_df['quantum_circuit']))
iqm_device = "Garnet"
shots = 50

# Run circuits on AWS linked RIgetti hardware
output = aws_run_task(key_circuit, iqm_device, shots)
for key, depth, task_arn in output:
    device = 'aws_'+iqm_device
    output_df.loc[len(output_df)] = [key, device, depth, task_arn, '', '']

In [22]:
# Print all output entries for AWS linked hardware
aws_df = output_df[output_df['device_name'].str.startswith(('aws'))]
aws_df

,key,device_name,transpiled_depth,job_id,counts,execution_time
30,e0dcdfc9,aws_local,12,,{'01': 1024},1970-01-01 00:00:00.122817278
31,23caeab0,aws_local,18,,"{'111': 982, '001': 13, '110': 6, '010': 7, '0...",1970-01-01 00:00:00.083653927
32,a00dc22c,aws_local,27,,"{'1001': 986, '0011': 3, '0000': 7, '1111': 5,...",1970-01-01 00:00:00.194475174
33,cdbc3c51,aws_local,38,,"{'10011': 481, '00111': 511, '01101': 3, '0100...",1970-01-01 00:00:00.600647450
34,0e14030c,aws_local,50,,"{'101001': 505, '011011': 518, '001110': 1}",1970-01-01 00:00:00.900839567
35,e0dcdfc9,aws_SV1,12,arn:aws:braket:us-west-1:615299756142:quantum-...,{'01': 1024},1970-01-01 00:00:07.659070253
36,23caeab0,aws_SV1,48,arn:aws:braket:us-west-1:615299756142:quantum-...,"{'111': 972, '011': 7, '001': 9, '010': 6, '00...",1970-01-01 00:00:04.601913452
37,a00dc22c,aws_SV1,186,arn:aws:braket:us-west-1:615299756142:quantum-...,"{'1001': 986, '0001': 3, '0011': 2, '1111': 4,...",1970-01-01 00:00:04.560971975
38,cdbc3c51,aws_SV1,530,arn:aws:braket:us-west-1:615299756142:quantum-...,"{'00111': 505, '10011': 482, '11000': 3, '1111...",1970-01-01 00:00:05.362342834
39,0e14030c,aws_SV1,1206,arn:aws:braket:us-west-1:615299756142:quantum-...,"{'101001': 522, '011011': 501, '110001': 1}",1970-01-01 00:00:06.537157297


In [23]:
print("Quantum Task Summary")
print(t.quantum_tasks_statistics())
print(
    "Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing "
    "unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated "
    "charges do not factor in any discounts or credits, and you may experience additional charges "
    "based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2)."
)
print(
    f"Estimated cost to run simulator tasks: {t.simulator_tasks_cost():.3f} USD"
)
print(
    f"Estimated cost to run the entire task: {t.qpu_tasks_cost() + t.simulator_tasks_cost():.3f} USD"
)

Quantum Task Summary
{'arn:aws:braket:us-east-1::device/qpu/ionq/Forte-1': {'shots': 300, 'tasks': {'CREATED': 3}}}
Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).
Estimated cost to run simulator tasks: 0.000 USD
Estimated cost to run the entire task: 24.900 USD


## 5. Collecting Results from Queued Jobs/Tasks <a name="results"></a>


In [7]:
# Print all output entries
output_df

,key,device_name,transpiled_depth,job_id,counts,execution_time
0,e0dcdfc9,aer_sim,6,,{'01': 1024},1970-01-01 00:18:06
1,23caeab0,aer_sim,10,,"{'111': 969, '001': 9, '110': 12, '011': 11, '...",1970-01-01 00:05:37
2,a00dc22c,aer_sim,14,,"{'1001': 986, '1101': 5, '0011': 3, '1100': 6,...",1970-01-01 00:03:21
3,cdbc3c51,aer_sim,20,,"{'00111': 505, '10011': 483, '10010': 2, '0110...",1970-01-01 00:05:24
4,0e14030c,aer_sim,26,,"{'011011': 507, '101001': 513, '111011': 1, '0...",1970-01-01 00:03:02
5,e0dcdfc9,aer_noisy,12,,"{'01': 950, '11': 49, '00': 19, '10': 6}",1970-01-01 04:16:30
6,23caeab0,aer_noisy,176,,"{'000': 28, '111': 788, '101': 37, '110': 38, ...",1970-01-01 03:22:52
7,a00dc22c,aer_noisy,659,,"{'1011': 40, '1001': 427, '0110': 37, '0111': ...",1970-01-01 02:24:09
8,cdbc3c51,aer_noisy,2332,,"{'10000': 32, '11111': 37, '00110': 40, '11010...",1970-01-01 03:27:55
9,0e14030c,aer_noisy,6315,,"{'000011': 19, '111110': 13, '100110': 14, '11...",1970-01-01 02:33:09


In [8]:
# Get results from real IBM backend
from ibm_qiskit.helper import update_dataframe_with_ibm_results

service = QiskitRuntimeService()
update_dataframe_with_ibm_results(service, output_df)

# Print results from IBM devices
ibm_df = output_df[output_df['device_name'].str.startswith(('aer', 'ibm'))]
ibm_df

Updated job cxb011mtpsjg008chm30: Counts = {'01': 989, '00': 13, '11': 17, '10': 5}
Updated job cxb012mrkac00089qwr0: Counts = {'011': 72, '101': 64, '001': 49, '110': 78, '111': 593, '000': 62, '100': 56, '010': 50}
Updated job cxb013mbqkhg008952sg: Counts = {'1001': 172, '0111': 55, '1010': 50, '1110': 53, '1011': 48, '1101': 56, '0110': 68, '1000': 61, '0101': 66, '1100': 50, '0100': 59, '0011': 69, '0010': 59, '1111': 36, '0001': 68, '0000': 54}
Updated job cxb014mpjw30008h89x0: Counts = {'00101': 38, '01101': 31, '10101': 33, '11110': 39, '01011': 25, '01010': 29, '00111': 32, '00110': 32, '00001': 36, '10001': 32, '00011': 30, '11111': 36, '11000': 39, '01110': 39, '10010': 23, '00100': 47, '10011': 31, '01100': 33, '11101': 30, '01000': 27, '10100': 29, '00000': 28, '01001': 36, '11100': 33, '10110': 29, '11010': 33, '01111': 31, '11001': 36, '00010': 24, '11011': 27, '10000': 26, '10111': 30}
Updated job cxb016crkac00089qwrg: Counts = {'010001': 18, '010101': 17, '011111': 20, 

,key,device_name,transpiled_depth,job_id,counts,execution_time
0,e0dcdfc9,aer_sim,6,,{'01': 1024},1970-01-01 00:18:06
1,23caeab0,aer_sim,10,,"{'111': 969, '001': 9, '110': 12, '011': 11, '...",1970-01-01 00:05:37
2,a00dc22c,aer_sim,14,,"{'1001': 986, '1101': 5, '0011': 3, '1100': 6,...",1970-01-01 00:03:21
3,cdbc3c51,aer_sim,20,,"{'00111': 505, '10011': 483, '10010': 2, '0110...",1970-01-01 00:05:24
4,0e14030c,aer_sim,26,,"{'011011': 507, '101001': 513, '111011': 1, '0...",1970-01-01 00:03:02
5,e0dcdfc9,aer_noisy,12,,"{'01': 950, '11': 49, '00': 19, '10': 6}",1970-01-01 04:16:30
6,23caeab0,aer_noisy,176,,"{'000': 28, '111': 788, '101': 37, '110': 38, ...",1970-01-01 03:22:52
7,a00dc22c,aer_noisy,659,,"{'1011': 40, '1001': 427, '0110': 37, '0111': ...",1970-01-01 02:24:09
8,cdbc3c51,aer_noisy,2332,,"{'10000': 32, '11111': 37, '00110': 40, '11010...",1970-01-01 03:27:55
9,0e14030c,aer_noisy,6315,,"{'000011': 19, '111110': 13, '100110': 14, '11...",1970-01-01 02:33:09


In [39]:
# Get results from AWS linked devices
from aws_braket.helper import update_dataframe_with_aws_results

update_dataframe_with_aws_results(output_df)

# Print results from AWS linked devices
aws_df = output_df[output_df['device_name'].str.startswith(('aws'))]
aws_df

Updated job arn:aws:braket:us-west-1:615299756142:quantum-task/4897ee66-79df-4238-b3f4-4e7000dbc75d: Counts = {'01': 1024}
Updated job arn:aws:braket:us-west-1:615299756142:quantum-task/2213e1ee-791b-4bb4-b736-193bf47ddcc7: Counts = {'111': 972, '011': 7, '001': 9, '010': 6, '000': 6, '110': 11, '101': 6, '100': 7}
Updated job arn:aws:braket:us-west-1:615299756142:quantum-task/9bca1d55-3496-49d7-a85b-6ffd8e440310: Counts = {'1001': 986, '0001': 3, '0011': 2, '1111': 4, '0000': 5, '1000': 3, '1010': 4, '0100': 4, '1011': 6, '0111': 3, '1110': 2, '0110': 1, '0010': 1}
Updated job arn:aws:braket:us-west-1:615299756142:quantum-task/3fabb08c-e1b6-4bdf-afdb-b14a34edb06c: Counts = {'00111': 505, '10011': 482, '11000': 3, '11110': 1, '00101': 3, '10000': 2, '00000': 1, '10101': 2, '00011': 2, '11111': 1, '01111': 1, '11100': 1, '10111': 2, '10110': 3, '11010': 4, '01011': 1, '11101': 1, '11001': 1, '01000': 1, '00010': 2, '10010': 1, '10001': 1, '01001': 1, '00100': 1, '01010': 1}
Updated job 

,key,device_name,transpiled_depth,job_id,counts,execution_time
30,e0dcdfc9,aws_local,12,,{'01': 1024},1970-01-01 00:00:00.122817278
31,23caeab0,aws_local,18,,"{'111': 982, '001': 13, '110': 6, '010': 7, '0...",1970-01-01 00:00:00.083653927
32,a00dc22c,aws_local,27,,"{'1001': 986, '0011': 3, '0000': 7, '1111': 5,...",1970-01-01 00:00:00.194475174
33,cdbc3c51,aws_local,38,,"{'10011': 481, '00111': 511, '01101': 3, '0100...",1970-01-01 00:00:00.600647450
34,0e14030c,aws_local,50,,"{'101001': 505, '011011': 518, '001110': 1}",1970-01-01 00:00:00.900839567
35,e0dcdfc9,aws_SV1,12,arn:aws:braket:us-west-1:615299756142:quantum-...,{'01': 1024},1970-01-01 00:00:07.659070253
36,23caeab0,aws_SV1,48,arn:aws:braket:us-west-1:615299756142:quantum-...,"{'111': 972, '011': 7, '001': 9, '010': 6, '00...",1970-01-01 00:00:04.601913452
37,a00dc22c,aws_SV1,186,arn:aws:braket:us-west-1:615299756142:quantum-...,"{'1001': 986, '0001': 3, '0011': 2, '1111': 4,...",1970-01-01 00:00:04.560971975
38,cdbc3c51,aws_SV1,530,arn:aws:braket:us-west-1:615299756142:quantum-...,"{'00111': 505, '10011': 482, '11000': 3, '1111...",1970-01-01 00:00:05.362342834
39,0e14030c,aws_SV1,1206,arn:aws:braket:us-west-1:615299756142:quantum-...,"{'101001': 522, '011011': 501, '110001': 1}",1970-01-01 00:00:06.537157297


## 6. Saving Data to CSV and JSON files<a name="save_data"></a>

In [9]:
# Creating Timestamps (Run only once for a given set of data)

timestamp = datetime.now().strftime('%Y%m%d_%H%M')

subfolder = 'saved_data'

# Create the subfolder if it does not exist
os.makedirs(subfolder, exist_ok=True)

# Create file names
param_csv = os.path.join(subfolder, f"parameters_{timestamp}.csv")
output_csv = os.path.join(subfolder, f"output_{timestamp}.csv")

param_json = os.path.join(subfolder, f"parameters_{timestamp}.json")
output_json = os.path.join(subfolder, f"output_{timestamp}.json")

In [11]:
# Save DataFrames to CSV files

# # Give file name to update existing files
param_csv = 'saved_data/parameters_20241208_1221.csv'
output_csv = 'saved_data/output_20241208_1221.csv'

param_df.to_csv(param_csv, index=False)  # index=False prevents extra index column
output_df.to_csv(output_csv, index=False)

In [10]:
# Save DataFrames to JSON files
import copy
from helper import get_circuit_list_for_json

# # Give file name to update existing files
param_json = 'saved_data/parameters_20241208_1221.json'
output_json = 'saved_data/output_20241208_1221.json'

store_df = copy.deepcopy(param_df)
store_df['quantum_circuit'] = get_circuit_list_for_json(store_df['quantum_circuit'])
store_df.to_json(param_json, orient='records')

output_df.to_json(output_json, orient='records')


----------------------------- End of Notebook ------------------------------